# Lecture 2: Introduction to Feature Engineering

Some comments have been added after the lecture 2 deadline. No code was changed.

My feature engineering consists of:
    count for each transaction how often the user already bought an item with the same garment group
    the percentage of the purchases of the user that have the same garment group as the transaction

Baseline has recall/precision/f1-score around 0.50, my version has about 0.75 for each metric.

## Setup

In [1]:
import numpy as np 
import pandas as pd
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [2]:
# Uncomment if using google colab
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Uncomment if using google colab, check data location
# %cd "/content/drive/MyDrive/data"
# !ls

/content/drive/MyDrive/data
articles.csv		   customers_sample5.csv.gz
articles_sample01.csv.gz   sample_submission.csv
articles_sample1.csv.gz    transactions_sample01.csv.gz
articles_sample5.csv.gz    transactions_sample1.csv.gz
customers.csv		   transactions_sample5.csv.gz
customers_sample01.csv.gz  transactions_train.csv
customers_sample1.csv.gz


In [4]:
articles = pd.read_csv('../data/articles.csv')
customers = pd.read_csv('../data/customers.csv')
sample_submisison = pd.read_csv('../data/sample_submission.csv')
transactions = pd.read_csv('../data/transactions_train.csv')

## The H&M Dataset

In [5]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

In [6]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355971 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 73.3+ MB


In [7]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28490064 entries, 0 to 28490063
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        int64  
 3   price             float64
 4   sales_channel_id  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.1+ GB


In [8]:
# X = transactions.merge(customers, how='inner', on='customer_id')
# X = X.merge(articles, how='inner', on='article_id')

### Creating Samples 
If you would rather work with samples instead of the whole dataset (while prototyping your code). You can use the code below:

In [9]:
# Adapted from: https://www.kaggle.com/code/paweljankiewicz/hm-create-dataset-samples
# This extracts three sampled datasets, containing 0.1%, 1% and 5% of all users and their transactions, and the associated articles.
for sample_repr, sample in [("01", 0.001), ("1", 0.01), ("5", 0.05)]:
    print(sample)
    customers_sample = customers.sample(int(customers.shape[0]*sample), replace=False)
    customers_sample_ids = set(customers_sample["customer_id"])
    transactions_sample = transactions[transactions["customer_id"].isin(customers_sample_ids)]
    articles_sample_ids = set(transactions_sample["article_id"])
    articles_sample = articles[articles["article_id"].isin(articles_sample_ids)]
    customers_sample.to_csv(f"../data/customers_sample{sample_repr}.csv.gz", index=False)
    transactions_sample.to_csv(f"../data/transactions_sample{sample_repr}.csv.gz", index=False)
    articles_sample.to_csv(f"../data/articles_sample{sample_repr}.csv.gz", index=False)

0.001
0.01
0.05


In [10]:
articles_sample = pd.read_csv('../data/articles_sample1.csv.gz')
customers_sample = pd.read_csv('../data/customers_sample1.csv.gz')
transactions_sample = pd.read_csv('../data/transactions_sample1.csv.gz')

In [11]:
customers_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13719 entries, 0 to 13718
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_id             13719 non-null  object 
 1   FN                      4864 non-null   float64
 2   Active                  4723 non-null   float64
 3   club_member_status      13648 non-null  object 
 4   fashion_news_frequency  13547 non-null  object 
 5   age                     13549 non-null  float64
 6   postal_code             13719 non-null  object 
dtypes: float64(3), object(4)
memory usage: 750.4+ KB


In [12]:
transactions_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284701 entries, 0 to 284700
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   t_dat             284701 non-null  object 
 1   customer_id       284701 non-null  object 
 2   article_id        284701 non-null  int64  
 3   price             284701 non-null  float64
 4   sales_channel_id  284701 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 10.9+ MB


## A Simplified Task: Binary Classification

The task of predicting which 12 items users are most likely to buy in the next week is difficult to translate to a traditional classification machine learning setting. 
To obtain the 12 items a user is most likely to buy, we need to make predictions for all items (or the ones selected by a baseline) and select the 12 that have the highest predicted scores.

In this assignment, we'll consider a simplified task: Predict whether a user ordered a single item or not, based on the features of the user and the item. 
We provide a baseline logistic regression model below, but haven't done much feature preprocessing or engineering!
Initially, it is always best to focus your efforts on getting your features in the right shape and setting up the right validation scheme and baselines.
Once you are sure that your features add value and your validation scheme is correct, then you typically move on to trying more elaborate models.

### Creating the Dataset

In [13]:
# If you'd rather use a sample. Uncomment the following code:
transactions = transactions_sample
customers = customers_sample
articles = articles_sample

In [14]:
transactions['ordered'] = 1

The problem setting is an example of a "PU learning" problem, i.e. only positives are labeled, everything else is unlabeled (and can be either positive or negative). 
Of course, we cannot train a classifier with just positive samples: The classifier will just learn that everything is positive.
Therefore, we need to manually generate negative samples.

Below, we use a simple random negative sampling strategy.
We want to create a balanced dataset, meaning that we have just as many positives as negatives.
This makes sure that the classifier will not benefit from predicting the positive/negative class more often than the other.
Realistically, the amount of positive samples is of course many times smaller than the amount of unlabeled, possibly negative instances.


If you want to try your hand at a more complex negative sampling strategy, you may want to check out this blog as a starting point: https://medium.com/mlearning-ai/overview-negative-sampling-on-recommendation-systems-230a051c6cd7.



In [15]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-20,00b1804e347ad55d1a75d7c07169e17d1890ad48953824...,294008002,0.013542,2.0,1
1,2018-09-20,016ee5d84576b72727d7c812df0080f8fa551ba7156593...,680186001,0.042356,1.0,1
2,2018-09-20,03748d7daaf2bd87ca7202935269c3220f335d98bd0bc1...,594987005,0.008881,2.0,1
3,2018-09-20,03748d7daaf2bd87ca7202935269c3220f335d98bd0bc1...,211143022,0.017780,2.0,1
4,2018-09-20,03748d7daaf2bd87ca7202935269c3220f335d98bd0bc1...,629551004,0.011847,2.0,1


In [16]:
# What's happening here? 
# We're creating negative samples. I.e. we're creating transactions that didn't actually occur.
# First, we need to know which interactions did occur:
positive_pairs = list(map(tuple, transactions[['customer_id', 'article_id']].drop_duplicates().values))

In [17]:
# Then we need to know what every synthetic transaction should contain: a date, a customer_id, an article_id, price, sales_channel_id. We will set ordered = 0, as these transactions didn't really occur.
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-20,00b1804e347ad55d1a75d7c07169e17d1890ad48953824...,294008002,0.013542,2.0,1
1,2018-09-20,016ee5d84576b72727d7c812df0080f8fa551ba7156593...,680186001,0.042356,1.0,1
2,2018-09-20,03748d7daaf2bd87ca7202935269c3220f335d98bd0bc1...,594987005,0.008881,2.0,1
3,2018-09-20,03748d7daaf2bd87ca7202935269c3220f335d98bd0bc1...,211143022,0.017780,2.0,1
4,2018-09-20,03748d7daaf2bd87ca7202935269c3220f335d98bd0bc1...,629551004,0.011847,2.0,1


In [18]:
# Extract real values
real_dates = transactions["t_dat"].unique()
real_customers = transactions["customer_id"].unique()
real_articles = transactions["article_id"].unique()
real_channels = transactions["sales_channel_id"].unique()
article_and_price = transactions[["article_id","price"]].drop_duplicates("article_id").set_index("article_id").squeeze()

In [19]:
# How many negatives do we need to sample?
num_neg_pos = transactions.shape[0]
print(num_neg_pos)

284701


In [20]:
# Sampling negatives by selecting random users, articles, dates and sales channel:
# Note: This is quite naive. Some articles may not even have been available at the date we are sampling.
random.seed(42)

# Afterwards, we need to remove potential duplicates, so we'll sample too many.
num_neg_samples = int(num_neg_pos * 1.1)

# Sample each of the independent attributes.
neg_dates = np.random.choice(real_dates, size=num_neg_samples)
neg_articles = np.random.choice(real_articles, size=num_neg_samples)
neg_customers = np.random.choice(real_customers, size=num_neg_samples)
neg_channels = np.random.choice(real_channels, size=num_neg_samples)
ordered = np.array([0] * num_neg_samples)
# Assign to every article a real price.
neg_prices = article_and_price[neg_articles].values

In [21]:
neg_transactions = pd.DataFrame([neg_dates, neg_customers, neg_articles, neg_prices, neg_channels, ordered], index=transactions.columns).T

In [22]:
# Result:
neg_transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-11-12,46df7910875b9d820e8d64662bdf59c1b61d39c24f1876...,647837001,0.084729,1.0,0
1,2019-09-28,3ac10a8573be6f4b78b3e93a0f590cc98ef06b1835a1ff...,556872002,0.036,1.0,0
2,2019-05-12,57034075e2c19d5e70ad8ca2c71c6d2404eeae54eb70a0...,733749004,0.004661,2.0,0
3,2020-01-05,c89285c3290476d6787c1d57f29ba5d6f3bf1b685f7e17...,704758027,0.012186,1.0,0
4,2019-06-03,d6ea05d4a6fa3fe40cc26b76b2322ce27a325bd0b38811...,562613010,0.022017,1.0,0


In [23]:
neg_transactions.shape

(313171, 6)

In [24]:
# Remove random negative samples that actually coincide with positives
df = neg_transactions[
    ~neg_transactions.set_index(["customer_id", "article_id"]).index.isin(positive_pairs)
]

# Remove any excess
chosen_neg_transactions = df.sample(num_neg_pos)

In [25]:
# Concat the negative samples to the positive samples:
transactions = pd.concat([transactions, chosen_neg_transactions])
transactions = transactions.merge(customers, how="inner", on='customer_id')
transactions = transactions.merge(articles, how="inner", on='article_id')

In [26]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 569402 entries, 0 to 569401
Data columns (total 36 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   t_dat                         569402 non-null  object 
 1   customer_id                   569402 non-null  object 
 2   article_id                    569402 non-null  object 
 3   price                         569402 non-null  object 
 4   sales_channel_id              569402 non-null  object 
 5   ordered                       569402 non-null  object 
 6   FN                            226771 non-null  float64
 7   Active                        221889 non-null  float64
 8   club_member_status            567376 non-null  object 
 9   fashion_news_frequency        563896 non-null  object 
 10  age                           564639 non-null  float64
 11  postal_code                   569402 non-null  object 
 12  product_code                  569402 non-nul

### Basic Preprocessing
Some very basic preprocessing.

In [27]:
# I'm dropping a lot of columns, use them in your engineering tasks!
transactions_processed = transactions[['customer_id', 'age', 'article_id', 'sales_channel_id', 'price', 'ordered']].copy()
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered
0,00b1804e347ad55d1a75d7c07169e17d1890ad48953824...,53.0,294008002,2.0,0.013542,1
1,bcee2c3ba37af84df42d121e5f0d10a1024232847f5b21...,26.0,294008002,1.0,0.016932,1
2,bcee2c3ba37af84df42d121e5f0d10a1024232847f5b21...,26.0,294008002,1.0,0.016932,1
3,3ef659a6e70080f59ffe0c2ba3e84525a3704693c748db...,49.0,294008002,1.0,0.013542,1
4,3ef659a6e70080f59ffe0c2ba3e84525a3704693c748db...,49.0,294008002,1.0,0.016932,1


In [28]:
# Does it make sense to label encode?
# Label encoding the customer and article IDs:
customer_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()

In [29]:
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])

In [30]:
# If you want to go back to the original encoding:
customer_encoder.inverse_transform([2])

array(['0016df4fbb49288b9ed4a8a0fa3d2f7038fc4ab7f02e3c68f5def1834a158340'],
      dtype=object)

In [31]:
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered
0,31,53.0,392,2.0,0.013542,1
1,9555,26.0,392,1.0,0.016932,1
2,9555,26.0,392,1.0,0.016932,1
3,3098,49.0,392,1.0,0.013542,1
4,3098,49.0,392,1.0,0.016932,1


In [32]:
# Can you come up with a NaN strategy that makes sense for each column in the dataset?
# Imputing all NaN values with zeros:
transactions_processed = transactions_processed.fillna(0)
transactions_processed.isnull().values.any()

False

In [33]:
# Does it make sense to one-hot encode?
# One-hot-encoding sales_channel_id:
transactions_processed = pd.get_dummies(transactions_processed, columns=['sales_channel_id'])

In [34]:
transactions_processed.head()

,customer_id,age,article_id,price,ordered,sales_channel_id_1.0,sales_channel_id_2.0
0,31,53.0,392,0.013542,1,0,1
1,9555,26.0,392,0.016932,1,1,0
2,9555,26.0,392,0.016932,1,1,0
3,3098,49.0,392,0.013542,1,1,0
4,3098,49.0,392,0.016932,1,1,0


In [35]:
# Creating a Train / Test Split:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)

In [36]:
X_train.head()

,customer_id,age,article_id,price,sales_channel_id_1.0,sales_channel_id_2.0
483378,8625,25.0,33665,0.013542,1,0
98596,6826,23.0,10457,0.040864,0,1
297135,8686,21.0,4555,0.025407,0,1
349729,10615,20.0,5168,0.008119,0,1
537827,6141,52.0,18804,0.061000,1,0


In [37]:
y_train.head()

483378    0
98596     1
297135    0
349729    0
537827    0
Name: ordered, dtype: int64

## Baseline Model

In [38]:
# Will take a few minutes to run, if you're using the whole dataset:
baseline = LogisticRegression(random_state=42)
baseline = baseline.fit(X_train, y_train)

In [39]:
baseline.predict_proba(X_test)

array([[0.49237345, 0.50762655],
       [0.51151456, 0.48848544],
       [0.50665147, 0.49334853],
       ...,
       [0.49695671, 0.50304329],
       [0.50482265, 0.49517735],
       [0.50168826, 0.49831174]])

In [40]:
y_test

227472    1
180528    1
317245    1
128006    0
531249    0
         ..
141939    1
75678     1
91293     1
342581    1
173686    1
Name: ordered, Length: 56941, dtype: int64

In [41]:
# Mean Accuracy:
baseline.score(X_test, y_test)
# As you can seen, the accuracy is ~0.51. In other words, the classifier predicts correctly 51% of the time whether a customer did or din't buy an item.
# Can you improve this baseline logistic regression model by doing better preprocessing and generating new features?
# Also think about my steps! Did it make sense to include the article and customer ids? (And things like that)

0.5098962083560177

In [42]:
# Classification Metrics:
predictions = baseline.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.51      0.68      0.58     28522
           1       0.51      0.34      0.41     28419

    accuracy                           0.51     56941
   macro avg       0.51      0.51      0.50     56941
weighted avg       0.51      0.51      0.50     56941



In [43]:
predictions

array([1, 0, 0, ..., 1, 0, 0])

## Assignment: Feature engineering
**TODO:** 
- In groups (of 2-3 students), think about (a few) features that can be engineered (preprocess and generate new features). Divide the work!
- Do these engineered features improve the baseline model?
- Add your thoughts & results to a slide deck for discussion next week (again, 1 slide per person).


In [45]:
# To get more information about articles compared to baseline, keep track of product type and garment group.
transactions_processed = transactions[['customer_id', 'age', 'article_id','garment_group_no','product_type_no', 'sales_channel_id', 'price', 'ordered']].copy()

# From https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html:
# "This transformer should be used to encode target values, i.e. y, and not the input X."
# However, I wasn't sure wwhat else to do here.
customer_encoder = preprocessing.LabelEncoder()
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
label_encoder = preprocessing.LabelEncoder()
transactions_processed['article_id'] = label_encoder.fit_transform(transactions_processed['article_id'])

# Age is the only column I selected that has null values. rather than setting
# it to 0 and considering everyone that didn't set his age to be a newborn baby, 
# it makes more sense to take the mean age to fill null values.
transactions_processed['age'] = transactions_processed['age'].fillna(transactions_processed['age'].mean())
# Check if other columns really don't have null values
transactions_processed.isnull().values.any()

# First groups by garment type. then for each garment type, count how many were ordered. reset_index makes a new column containing the counts.
temp_product_bought_percent_of_time = transactions_processed.groupby('garment_group_no')["ordered"].sum().reset_index(name="type_bought_no")
# Should have columns garment_group_no and type_bought_no
temp_product_bought_percent_of_time.head()

# Merge column created above into transaction data
transactions_processed = transactions_processed.merge(temp_product_bought_percent_of_time,how="inner",on="garment_group_no")

# For each user and garment group combination, calculate how often the user bought garments of that type.
# Only take into account the transactions where the item was indeed ordered.
temp_user_bought_percent_of_time = transactions_processed.groupby(['customer_id',"garment_group_no"]).size().reset_index(name="user_bought_no")
temp_user_bought_percent_of_time.head()

# To each transaction, add how often the user doing the transaction bought products of the garment type of the product.
transactions_processed = transactions_processed.merge(temp_user_bought_percent_of_time,how="inner",on=["garment_group_no","customer_id"])

# One-hot encode
transactions_processed = pd.get_dummies(transactions_processed, columns=['sales_channel_id','garment_group_no','product_type_no'])
standardscaling = preprocessing.StandardScaler()
# One-hot encoding creates a bunch of new columns, so instead of selecting 
# existing columns, select all and remove the ones that do not need scaling.
columns = transactions_processed.columns.tolist()
columns.remove('article_id')
columns.remove('customer_id')
columns.remove('ordered')
transactions_processed[columns] = standardscaling.fit_transform(transactions_processed[columns])

# article_id and customer_id could have been useful for merging in the code 
# above, but training without them seems to give better results somehow.
# I expected that it would be better to keep customer ids: if someone bought a 
# certain product type several times, I expected the regression model to learn
# from that and make more accurate predictions for that user.
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop(['ordered','article_id','customer_id'], axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)
y_train = y_train.astype('int')
y_test = y_test.astype('int')

# Max_iter increase ensures that the program won't crash. It's still rather fast.
engineered = LogisticRegression(random_state=42,max_iter=10000)
engineered = engineered.fit(X_train, y_train)

engineered.predict_proba(X_test)
engineered.score(X_test, y_test)
print(engineered.score(X_test, y_test))

predictions = engineered.predict(X_test)
print(classification_report(y_test, predictions))

0.7503029451537556
              precision    recall  f1-score   support

           0       0.72      0.83      0.77     28532
           1       0.80      0.67      0.73     28409

    accuracy                           0.75     56941
   macro avg       0.76      0.75      0.75     56941
weighted avg       0.76      0.75      0.75     56941

